In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

from awrams.utils import extents
from awrams.utils import datetools as dt
from awrams.utils.nodegraph import nodes
from awrams.simulation import ondemand

# The AWRA-CMS Configuration System

The purpose of this notebook is to demonstrate the AWRA-CMS configuration system. You will need to understand this configuration to setup, customise and run simulations on your system. The notebook has the following sections:
1. Model configuration
2. System configuration
3. Small scale changes to model and system profile
4. Large scale changes to model and system profile

The AWRA-CMS has the flexibility to be run with multiple versions of AWRA-L models on both Windows and Linux systems. It uses a "model profile" to define the model to run, and a "system profile" to specify system specific settings to use when running it. The intent of this design is to allow you to easily share results by only requiring your notebook, model profile and perhaps system profile to be shared for others to reproduce your results.

The interface for interacting with these profiles is the `config_manager` module: 

In [ ]:
from awrams.utils import config_manager

Model and system profiles are Python modules that the `config_manager` looks for in `${AWRAMS_BASE_PATH}/config/models` and `${AWRAMS_BASE_PATH}/config/system` respectively. `AWRAMS_BASE_PATH` is an environment variable that should be set by the activation script used to activate the `awra-cms` environment. The `config_manager` can be used to see what `AWRAMS_BASE_PATH` is currently set to:

In [ ]:
base_path = config_manager.get_awrams_base_path()
str(base_path)

It can also be used to set `AWRAMS_BASE_PATH` from your notebook:

In [ ]:
config_manager.set_awrams_base_path(str(base_path))

## Model configuration

Model profiles are used to control:
- Which `awrams.models` class is used.
- What parameters the model is run with (e.g fixed and calibrated).
- What inputs the model needs.
- What values to use for those inputs (e.g constant value, computed values, spatio-temporal data from files, etc).
- What to use for initial states.
- What variables the model will output.

They can be retrieved by passing a model type and version to the `config_manager`'s `get_model_profile` method, which will load `${AWRAMS_BASE_PATH}/config/models/[TYPE]/[VERSION].py`:

In [ ]:
# The model type and version arguments are optional and we've used the default values here.
# Generally the AWRA-CMS is run with only 'awral' type models but has been designed to accomodate other model types.
model_type = 'awral'

# This is the most recent AWRA-L model version, but a v5 profile also comes packaged with the AWRA-CMS.
model_version = 'v6_default'  

model_profile = config_manager.get_model_profile(model_type, model_version)

In [ ]:
# NOTE: System specific file paths appearing below are actually pulled from the system profile by 'get_settings'.
model_settings = model_profile.get_settings()
model_settings

As `model_settings` is a nested Python dictionary we can access the individual elements, e.g.:

In [ ]:
model_settings['CLIMATE_DATASET']['FORCING']['PATH']

To run the model we will need the input mapping and we can get the model input map using the model settings:

In [ ]:
input_map = model_profile.get_input_mapping(model_settings)
input_map

As the input map is a Python dictionary we can access individual elements using their corresponding keys:

In [ ]:
input_map['precip_f']

If we can change an element of the `model_settings` we will need to feed this back into the model before setting up the input map to ensure that our changes are made before running the model:

In [ ]:
model_settings['CLIMATE_DATASET']['FORCING']['PATH'] = '/new/path/to/forcing'
input_map = model_profile.get_input_mapping(model_settings)

We can then confirm that the path to forcing has changed in the input mapping:

In [ ]:
input_map['precip_f']

### Exercises:
1. Retrieve the `MODEL_VERSION` from the `model_settings` dictionary by specifying its key in square brackets *Note that as `model settings` is a nested dictionary you will need two sets of square brackets.
2. Change the path to the spatial file in `model_settings` i.e. `['SPATIAL_FILE']` and update the `input_map`. Confirm that he path has changed as you expect.

## System configuration

System profiles are intended to contain all settings that are specific to the system we are running on. For general AWRA-CMS users, the most important thing to note is that **system profiles are where you should specify the location of files on the local filesystem that are needed by the AWRA-CMS**.

For more advanced users and system administrators, system profiles also provide control over:
- MPI configuration (e.g choice of whether to use `mpirun`, `aprun`, etc for launching parallel computations).
- IO settings (e.g chunking of input and output data).
- Logger settings.

The system profile used by the AWRA-CMS is controlled by the `AWRAMS_SYSTEM_PROFILE` environment variable. When running, the AWRA-CMS will load `${AWRAMS_BASE_PATH}/config/system/${AWRAMS_SYSTEM_PROFILE}.py` to look up system settings.

The `config_manager` allows you to set the active system profile in your notebooks using the `set_active_system_profile` method, and to inspect the current system profile using the `get_system_profile` method:

In [ ]:
profile_name = 'default'

# Under-the-hood, this just sets the AWRAMS_SYSTEM_PROFILE environment variable.
config_manager.set_active_system_profile(profile_name)

# The profile name argument is optional and will default to whatever the AWRAMS_SYSTEM_PROFILE environment variable is
# or to 'default' if that's not set.
system_profile = config_manager.get_system_profile(profile_name)

You can make changes to system settings in your notebooks by first retrieving the system settings from the system profile:

In [ ]:
system_settings = system_profile.get_settings()
system_settings

As `system_settings` is a nested Python dictionary we can access the individual elements, e.g.:

In [ ]:
system_settings['DATA_PATHS']['CATCHMENT_SHAPEFILE']

If we change elements of the `system_settings` though, **we have to feed this back to the model profile before running our simulation, calibration, etc**.

In [ ]:
system_settings['DATA_PATHS']['CATCHMENT_SHAPEFILE'] = '/new/path/to/shapefile/Final_list_all_attributes.shp'
system_settings
model_settings = model_profile.get_settings(system_settings)
model_settings

### Exercise:
Retrieve  the climate datasets training forcing path from the `system_settings` dictionary

## Small scale changes to model and system profile

For pedalogical purposes we introduce `model_settings` before `system_settings` but in AWRA-CMS workflows you will need to load the system settings before the model settings.

**Example workflow:**

In [ ]:
from awrams.utils import config_manager

# 1) Load system profile and settings
system_profile = config_manager.get_system_profile('default')
system_settings = system_profile.get_settings()

# 2) Make changes to the system settings
#system_settings['DATA_PATHS']['CATCHMENT_SHAPEFILE'] = '/new/path/to/shapefile/Final_list_all_attributes.shp'

# 3) Pass system settings to the model profile to return model settings that incorporate the change in system settings 
model_profile = config_manager.get_model_profile('awral', 'v6_default')
model_settings = model_profile.get_settings(system_settings)

# 4) Make model settings changes
#model_settings['CLIMATE_DATASET']['FORCING']['PATH'] = '/new/path/to/forcing'

# 5) Extract the input map from the updated model settings
input_map = model_profile.get_input_mapping(model_settings)

# 6) Make any additional changes to the input_map, e,g. fix the scale for effective porosity:
input_map['ne_scale'] = nodes.parameter(fixed=True, value=0.5, max_val=1, min_val=0.001, description='Scale for effective porosity - fixed to 0.5')

# 7) Use the get_model method to instantiate the model with the model settings 
awral_model = model_profile.get_model(model_settings)

# 8) Get an instance of the model runner engine - passing in the model (awral and input_map defined earlier)
runner = ondemand.OnDemandSimulator(awral_model, input_map)

# 9) Run the on demand simulation
my_period = dt.dates('jan 2010 - dec 2010')
def_ext = extents.get_default_extent()
my_extent = def_ext.factory.get_by_cell_coords(-36.75, 144.30)
results, inputs = runner.run(my_period, my_extent, True, False)

In [ ]:
plt.figure(figsize=(16, 3))
plt.plot(results['qtot'], label='qtot: Runoff')
plt.plot(results['etot'], label='etot : Actual evapotranspiration')
plt.plot(results['e0'], label='e0 : Potential evaporation')
plt.plot(results['dd'], label='dd : Deep drainage')
plt.xlabel('Day')
plt.ylabel('Flux [mm]')
plt.legend()

## Large scale changes to model and system profile

If you would like to change a small number of aspects of the system profile you can do this inside the notebook. However, if you would like to make larger scale changes to model and system profiles then we recommend that you create a new config file in `$AWRAMS_BASE_PATH/config/models` and/or `$AWRAMS_BASE_PATH/config/system` respectively.

***Note: Importantly creating new model and system configuration files in this manner allows you to share them with other AWRA-CMS users who wish to recreate your model and system setup.***

### Exercise:
1. Create a new model config file in `$AWRAMS_BASE_PATH/config/models` and load this model profile in in the notebook. 

**Hint:** make a copy of `$AWRAMS_BASE_PATH/config/models/awral/v6_default.py` say `$AWRAMS_BASE_PATH/config/models/awral/v6_training.py`, make a change to a data path in `v6_training.py` then load this up using `config_manager.get_model_profile('awral', 'v6_training')`.

2. Create a new system config file in `$AWRAMS_BASE_PATH/config/system` and load this model profile in the notebook. 

## Now onto a Python version of the AWRA-L code for demonstration....

#### [1.4 AWRA-L deconstructed - Python version of AWRA-L v6]
[1.4 AWRA-L deconstructed - Python version of AWRA-L v6]: 1.4_AWRA-L_deconstructed_-_Python_version_of_AWRA-L_v6.ipynb